# Exercise: Flood Analysis in Opole (2024) using Sentinel-1 SAR Data

In this exercise, you will compare radar backscatter from Sentinel-1 before and after the 2024 flood in Opole, Poland.

**Objective:**
- Load Sentinel-1 VV data before and after the flood
- Convert radar intensity to decibels (dB)
- Calculate the difference between the two dates
- Identify areas with a significant decrease in VV (possible flood zones)

## Step 1: Import Earth Engine and geemap

In [2]:
import ee
import geemap

ee.Initialize(project='ee-dabrowaremote')
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
region = ee.Geometry.Point([17.931, 50.675])

## Step 2: Load Sentinel-1 VV images (before and after flood)

In [ ]:
# TODO: Load Sentinel-1 collection for before and after flood
# Use filterBounds, filterDate, and select 'VV' band
# Apply .mean() to get average image from each period


In [3]:
# Define date ranges for before and after the flood
before_start = '2024-07-01'
before_end = '2024-08-20'
after_start = '2024-09-13'
after_end = '2024-10-16'

# Load Sentinel-1 collection, filter by region and date, select 'VV' band, and compute mean
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
    .filter(ee.Filter.eq('resolution_meters', 10))

before = s1_collection.filterDate(before_start, before_end).select('VV').mean()
after = s1_collection.filterDate(after_start, after_end).select('VV').mean()

In [4]:
before

In [5]:
after

In [8]:
# Ustaw parametry wizualizacji dla obrazu VV
vis_params = {
    'min': -20,
    'max': 0,
    'palette': ['blue', 'white', 'green']
}

# Dodaj warstwy "before" i "after" do mapy
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
Map.addLayer(before, vis_params, 'Sentinel-1 VV Before Flood')
Map.addLayer(after, vis_params, 'Sentinel-1 VV After Flood')
Map

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Step 3: Convert to dB scale

In [ ]:
# TODO: Use .log10().multiply(10) to convert before/after images to dB


In [7]:
before_dB = before.log10().multiply(10)
after_dB = after.log10().multiply(10)

In [10]:
# Ustaw parametry wizualizacji dla obrazu VV
vis_params = {
    'min': -25,
    'max': 0,
    'palette': ['blue', 'white', 'green']
}

# Dodaj warstwy "before" i "after" do mapy
Map2 = geemap.Map(center=[50.675, 17.931], zoom=12)
Map2.addLayer(before_dB, vis_params, 'Sentinel-1 VV dB Before Flood')
Map2.addLayer(after_dB, vis_params, 'Sentinel-1 VV dB After Flood')
Map2

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Step 4: Calculate difference and threshold potential flood areas

In [ ]:
# TODO: Subtract after_dB from before_dB
# Create flood mask: pixels where difference > 3 dB

In [11]:
# Calculate the difference in dB (before - after)
diff_dB = before_dB.subtract(after_dB)

# Create a flood mask: areas where the decrease is greater than 3 dB
flood_mask = diff_dB.gt(3)

## Step 5: Visualize the results

In [ ]:
# TODO: Use Map.addLayer() to show before, after, difference, and flood mask



In [12]:
Map3 = geemap.Map(center=[50.675, 17.931], zoom=12)
Map3.addLayer(before, vis_params, 'Sentinel-1 VV Before Flood')
Map3.addLayer(after, vis_params, 'Sentinel-1 VV After Flood')
Map3.addLayer(diff_dB, {'min': -10, 'max': 10, 'palette': ['red', 'white', 'green']}, 'Difference (dB)')
Map3.addLayer(flood_mask.updateMask(flood_mask), {'palette': ['blue']}, 'Flood Mask')
Map3

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…